# 🌍 AI Travel Agent with Microsoft Agent Framework (.NET)

## 📋 Scenario Overview

This notebook demonstrates how to build an intelligent travel planning agent using the Microsoft Agent Framework for .NET. The agent can automatically generate personalized day-trip itineraries for random destinations around the world.

**Key Capabilities:**
- 🎲 **Random Destination Selection**: Uses a custom tool to pick vacation spots
- 🗺️ **Intelligent Trip Planning**: Creates detailed day-by-day itineraries
- 🔄 **Real-time Streaming**: Supports both immediate and streaming responses
- 🛠️ **Custom Tool Integration**: Demonstrates how to extend agent capabilities

## 🔧 Technical Architecture

### Core Technologies
- **Microsoft Agent Framework**: Latest .NET implementation for AI agent development
- **GitHub Models Integration**: Uses GitHub's AI model inference service
- **OpenAI API Compatibility**: Leverages OpenAI client libraries with custom endpoints
- **Secure Configuration**: Environment-based API key management

### Key Components
1. **AIAgent**: The main agent orchestrator that handles conversation flow
2. **Custom Tools**: `GetRandomDestination()` function available to the agent
3. **Chat Client**: GitHub Models-backed conversation interface
4. **Streaming Support**: Real-time response generation capabilities

### Integration Pattern
```mermaid
graph LR
    A[User Request] --> B[AI Agent]
    B --> C[GitHub Models API]
    B --> D[GetRandomDestination Tool]
    C --> E[Travel Itinerary]
    D --> E
```

## 🚀 Getting Started

**Prerequisites:**
- .NET 9.0 or higher
- GitHub Models API access token
- Environment variables configured in `.env` file

**Required Environment Variables:**
```env
GITHUB_TOKEN=your_github_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Run the cells below in sequence to see the travel agent in action!

In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI
// This provides the core AI abstraction layer for .NET applications
// Essential for building AI agents with standardized interfaces
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// 🔗 Local Reference: OpenAI Agent Extensions
// This references the local build of the Microsoft Extensions AI Agents OpenAI library
// Contains OpenAI-specific implementations for the agent framework
// Required for integrating with OpenAI models and GitHub Models
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0\Microsoft.Agents.AI.OpenAI.dll"

In [3]:
// 🤖 Local Reference: Core Agent Framework
// This references the core Microsoft Extensions AI Agents library
// Provides the fundamental agent abstractions and base classes
// Essential for creating and managing AI agents
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [4]:
// 🔌 NuGet Package Reference: OpenAI Integration
// Preview version of Microsoft Extensions AI OpenAI integration
// Provides OpenAI client and service implementations
// Enables communication with OpenAI API and GitHub Models
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [5]:
// 🔧 NuGet Package Reference: Environment Variables
// DotNetEnv allows loading environment variables from .env files
// Essential for managing API keys and configuration securely
// Helps keep sensitive information out of source code
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
// 📚 Import Required Namespaces
using System;                          // Core system functionality and base types
using System.ComponentModel;           // For Description attributes on tool functions
using System.ClientModel;             // Client model abstractions for API interactions

using Microsoft.Extensions.AI;        // Core AI abstractions and interfaces
using Microsoft.Agents.AI;           // Agent framework classes and interfaces
using OpenAI;                         // OpenAI client library for API communication

In [7]:
// 🔧 Import Environment Variable Loading
// DotNetEnv namespace for loading .env files
// Enables secure configuration management from external files
using DotNetEnv;

In [8]:
// 🔧 Load Environment Variables from .env File
// This loads configuration from a .env file located 3 directories up
// Required variables: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
// Ensures secure handling of API credentials and configuration
Env.Load("../../../.env");

In [9]:
// 🎲 Tool Function: Random Destination Generator
// This static method will be available to the agent as a callable tool
// The [Description] attribute helps the AI understand when to use this function
// This demonstrates how to create custom tools for AI agents
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    // List of popular vacation destinations around the world
    // The agent will randomly select from these options
    var destinations = new List<string>
    {
        "Paris, France",         // European cultural capital
        "Tokyo, Japan",          // Asian modern metropolis
        "New York City, USA",    // American urban experience
        "Sydney, Australia",     // Oceanic coastal beauty
        "Rome, Italy",           // Historical European city
        "Barcelona, Spain",      // Mediterranean cultural hub
        "Cape Town, South Africa", // African scenic destination
        "Rio de Janeiro, Brazil",  // South American beach city
        "Bangkok, Thailand",        // Southeast Asian cultural center
        "Vancouver, Canada"         // North American natural beauty
    };

    // Generate random index and return selected destination
    // Uses System.Random for simple random selection
    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
// 🔑 Extract Configuration from Environment Variables
// Retrieve GitHub Models API endpoint, throws exception if not set
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
// Get the model ID, defaults to gpt-4o-mini if not specified
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
// Retrieve GitHub token for authentication, throws exception if not set
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
// ⚙️ Configure OpenAI Client Options
// Create configuration options to point to GitHub Models endpoint
// This redirects OpenAI client calls to GitHub's model inference service
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)  // Set custom endpoint for GitHub Models
};

In [12]:
// 🔌 Initialize OpenAI Client with GitHub Models Configuration
// Create OpenAI client using GitHub token for authentication
// Configure it to use GitHub Models endpoint instead of OpenAI directly
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
// 🤖 Create AI Agent with Travel Planning Capabilities
// Initialize OpenAI client, get chat client for specified model, and create AI agent
// Configure agent with travel planning instructions and random destination tool
// The agent can now plan trips using the GetRandomDestination function
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions)
    .GetChatClient(github_model_id)  // Get chat client for the specified model
    .CreateAIAgent(
        instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", 
        tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]  // Register the tool function
    );

In [14]:
// 🚀 Execute Agent: Plan a Day Trip (Non-Streaming)
// Run the agent with a travel planning request
// The agent will use the GetRandomDestination tool and create a complete itinerary
// Returns the full response once processing is complete
Console.WriteLine(await agent.RunAsync("Plan me a day trip"));

How about a lovely day trip to Rome, Italy? Here's a suggested itinerary for a perfect day:

### Morning:
- **Colosseum**: Start your day early by visiting the iconic Colosseum. Make sure to book your tickets in advance to avoid long lines.
- **Roman Forum**: After the Colosseum, take a stroll through the Roman Forum and soak in the ancient ruins and history.

### Lunch:
- **Trattoria**: Enjoy authentic Italian cuisine at a local trattoria. Try pasta dishes like Carbonara or Cacio e Pepe, paired with a nice glass of Italian wine.

### Afternoon:
- **Vatican City**: Spend your afternoon exploring Vatican City. Visit St. Peter's Basilica and the Vatican Museums, where you can marvel at the Sistine Chapel.
- **Piazza Navona**: Take a leisurely walk to Piazza Navona, one of Rome's most beautiful squares. Enjoy the fountains and street artists.

### Evening:
- **Dinner**: Head to a nearby restaurant for dinner and enjoy traditional Roman dishes.
- **Gelato**: End your evening with a delicio

In [15]:
// 🌊 Execute Agent: Plan a Day Trip (Streaming Response)
// Run the agent with streaming enabled for real-time response display
// Shows the agent's thinking and response as it generates the content
// Provides better user experience with immediate feedback
await foreach (var update in agent.RunStreamingAsync("Plan me a day trip"))
{
    Console.Write(update);  // Display each part of the response as it arrives
}

How about a day trip to Sydney, Australia? Here’s a suggested itinerary for you:

### Morning:
- **Breakfast at a Local Café:** Start your day with a hearty breakfast at one of Sydney’s famous cafés in The Rocks area. Try some avocado toast or a classic Aussie brekkie.
- **Explore the Sydney Opera House:** Take a guided tour of this iconic landmark. Don’t forget to snap some photos!

### Late Morning:
- **Stroll through the Royal Botanic Garden:** Enjoy a peaceful walk through the beautiful gardens and admire the views of the harbor.

### Lunch:
- **Lunch at Circular Quay:** Grab some fresh seafood or fish and chips from one of the many restaurants with a view of the harbor.

### Afternoon:
- **Visit the Art Gallery of New South Wales:** Explore the extensive collection of Australian and international art.
- **Take a Ferry to Manly Beach:** Enjoy the stunning views of the Sydney skyline from the water. 

### Evening:
- **Relax at Manly Beach:** Spend some time on the beach, enjoy the s